<a href="https://colab.research.google.com/github/movi-data/painel_performance/blob/main/painel_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relatório Plataformas

In [4]:
import pandas as pd

# --- ETAPA 1: Carregar o arquivo ---
df_propostas = pd.read_csv("propostas.csv", sep=";", encoding="utf-8", low_memory=False)

# --- ETAPA 2: Preparar Colunas ---

# Tranformar 'Canal' em maiuscula
df_propostas['Canal'] = df_propostas['Canal'].str.upper()
# Filtrar o primeiro Canal na coluna 'Canal'
df_propostas['Canal'] = df_propostas['Canal'].str.split(',').str[0]

# --- ETAPA 3: Fazer agrupamento e ordernar ---

# Agrupar leads por mês na coluna 'Data' e 'Canal'
leads_por_canal_mes = df_propostas.groupby([pd.to_datetime(df_propostas["Data"], format="%d/%m/%Y", errors='coerce').dt.to_period("d"),
                                            "Canal"]).size().reset_index(name="Quantidade de Leads")


In [5]:
# --- ETAPA 4: Criar arquivo csv e download ---

# Criar arquivo .csv
leads_por_canal_mes.to_csv("leads_por_canal_mes.csv",sep=";" ,index=False)

# Download do arquivo
from google.colab import files
files.download("leads_por_canal_mes.csv")

# Vizualização para conferência
leads_por_canal_mes.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Data,Canal,Quantidade de Leads
0,2025-06-01,FACEBOOK,330
1,2025-06-01,INSTAGRAM,1330
2,2025-06-01,LOCAÇÃO,24
3,2025-06-01,LOJA,32
4,2025-06-01,MERCADO LIVRE,35


In [3]:
leads_por_canal_mes.tail()

,Data,Canal,Quantidade de Leads
372,2025-07-03,MOBIAUTO,41
373,2025-07-03,OLX,168
374,2025-07-03,SITE,69
375,2025-07-03,WEBMOTORS,66
376,2025-07-03,WHATSAPP,700


In [8]:
df_propostas.columns

Index(['ID Lead', 'Data', 'Data ultima interação', 'Nome PDV', 'CNPJ PDV',
       'Nome Cliente', 'Email Cliente', 'Telefone Cliente', 'CPF', 'Lead',
       'Origem', 'Canal', 'Status', 'CallCenter status', 'Data Status Loja',
       'Nome Vendedor', 'ID do Vendedor', 'Callcenter detalhe',
       'callcenter status detalhe', 'assistente vendas',
       'ID do Assistente de Vendas', 'tipo', 'qualificado', 'documentos',
       'tags', 'vetor_id', 'data_ult_solicitacao_agendamento',
       'data_ult_agendamento', 'agendamento_visita', 'atraso_ult_agendamento',
       'qtd_agendamentos_90d', 'qtd_nao_foi_90d', 'regiao', 'Cidade', 'UF',
       'CEP', 'campanha'],
      dtype='object')

In [9]:
# Valores unicos na coluna 'tipo'
df_propostas['tipo'].unique()

array(['PERFORMANCE', 'CLASSIFICADOS', 'LOJA', 'CRM'], dtype=object)

In [6]:
# Valores unicos na coluna 'Canal'
df_propostas['Canal'].unique()

array(['WHATSAPP', 'INSTAGRAM', 'OLX', 'FACEBOOK', 'SITE', 'WEBMOTORS',
       'MERCADO LIVRE', 'LOCAÇÃO', 'MOBIAUTO', 'LOJA', nan,
       'CAMPANHA DIGITAL', 'USADOS BR', 'AUTOLINE', 'AMIGO SAGA',
       'AUTOCARRO', 'CHAVES NA MÃO', 'NA PISTA', 'NA PISTA V2',
       'LIVETREE', 'AUTOSCAR', 'MOVIDA SEMINOVOS'], dtype=object)

In [7]:
# Valores unicos na coluna 'Origem'
df_propostas['Origem'].unique()

array(['MobiBot', 'Internet', 'Passante', 'CRM', 'Prospecção',
       'Ligação Receptiva', 'Indicação', 'desconhecido',
       'OLX Pessoa Física', 'Outros', 'Feirao'], dtype=object)

# Completo

In [37]:
import pandas as pd

# --- ETAPA 1: Carregar os arquivos ---
df_propostas = pd.read_csv("propostas.csv", sep=";", encoding="utf-8", low_memory=False)
df_lojas = pd.read_excel("de_para_loja.xlsx")

In [38]:
df_propostas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391194 entries, 0 to 391193
Data columns (total 37 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ID Lead                           391194 non-null  int64  
 1   Data                              391194 non-null  object 
 2   Data ultima interação             391194 non-null  object 
 3   Nome PDV                          391194 non-null  object 
 4   CNPJ PDV                          391194 non-null  int64  
 5   Nome Cliente                      387770 non-null  object 
 6   Email Cliente                     243051 non-null  object 
 7   Telefone Cliente                  390938 non-null  object 
 8   CPF                               10852 non-null   float64
 9   Lead                              51898 non-null   object 
 10  Origem                            391194 non-null  object 
 11  Canal                             376404 non-null  o

In [44]:
# --- ETAPA : Preparar Colunas ---

# Criar coluna 'Canal 2' com 'Canal' pegando o primeiro
df_propostas['Canal_2'] = df_propostas['Canal'].str.split(',').str[0]

# Criar coluna 'Bot abandonado'
df_propostas['Bot Abandonado'] = df_propostas['tags'].str.contains('Bot Abandonado', case=False)

In [45]:
df_propostas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391194 entries, 0 to 391193
Data columns (total 40 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ID Lead                           391194 non-null  int64  
 1   Data                              391194 non-null  object 
 2   Data ultima interação             391194 non-null  object 
 3   Nome PDV                          391194 non-null  object 
 4   CNPJ PDV                          391194 non-null  int64  
 5   Nome Cliente                      387770 non-null  object 
 6   Email Cliente                     243051 non-null  object 
 7   Telefone Cliente                  390938 non-null  object 
 8   CPF                               10852 non-null   float64
 9   Lead                              51898 non-null   object 
 10  Origem                            391194 non-null  object 
 11  Canal                             376404 non-null  o

In [42]:
# Valores unicos na coluna 'Canal 2'
df_propostas['Canal_2'].unique()

array(['WhatsApp', 'Instagram', 'OLX', 'Facebook', 'Site', 'Webmotors',
       'Mercado Livre', 'Locação', 'Mobiauto', 'Loja', nan,
       'Campanha Digital', 'Usados BR', 'Autoline', 'Amigo Saga',
       'Autocarro', 'Chaves na Mão', 'Na Pista', 'Na Pista V2',
       'LiveTree', 'Autoscar', 'Movida Seminovos'], dtype=object)

In [43]:
# Valores unicos na coluna 'tipo'
df_propostas['tipo'].unique()

array(['PERFORMANCE', 'CLASSIFICADOS', 'LOJA', 'CRM'], dtype=object)

In [47]:
# Criar coluna 'tipo 2' baseado no 'Canal_2'

'''
'WhatsApp', 'Instagram', 'OLX', 'Facebook', 'Site', 'Webmotors',
       'Mercado Livre', 'Locação', 'Mobiauto', 'Loja', nan,
       'Campanha Digital', 'Usados BR', 'Autoline', 'Amigo Saga',
       'Autocarro', 'Chaves na Mão', 'Na Pista', 'Na Pista V2',
       'LiveTree', 'Autoscar', 'Movida Seminovos'
'''


def tipo_2(Canal_2):
    if 'WhatsApp' in 'Canal_2':
        return 'WhatsApp'
    elif 'Facebook' in 'Canal_2':
        return 'Performance'
    elif 'Instagram' in 'Canal_2':
        return 'Performance'
    elif 'Site' in 'Canal_2':
        return 'Site'
    elif 'Loja' in 'Canal_2':
        return 'Loja'
    elif 'OLX':
        return 'Classificado'
    elif 'Mercado Livre' in 'Canal_2':
        return 'Classificado'
    elif 'Webmotors' in 'Canal_2':
        return 'Classificado'
    elif 'Usados BR' in 'Canal_2':
        return 'Classificado'
    elif 'Na Pista' in 'Canal_2':
        return 'Classificado'
    elif 'Na Pista V2' in 'Canal_2':
        return 'Classificado'
    elif 'Mobiauto' in 'Canal_2':
        return 'Classificado'
    elif 'Chaves na Mão' in 'Canal_2':
        return 'Classificado'
    else:
        return 'Outros'

In [48]:
# Manter colunas necessárias que não geram duplicidade
df_propostas = df_propostas[['ID Lead', 'Data', 'Data ultima interação', 'Nome PDV',
       'Nome Cliente', 'Email Cliente', 'Telefone Cliente', 'CPF', 'Lead',
       'Origem', 'Canal','CallCenter status', 'Data Status Loja',
       'Nome Vendedor', 'ID do Vendedor', 'Callcenter detalhe',
       'callcenter status detalhe', 'assistente vendas',
       'ID do Assistente de Vendas', 'tipo', 'qualificado', 'documentos',
       'tags', 'vetor_id', 'data_ult_solicitacao_agendamento',
       'data_ult_agendamento', 'agendamento_visita', 'atraso_ult_agendamento',
       'qtd_agendamentos_90d', 'qtd_nao_foi_90d', 'regiao','campanha',
        'Canal 2', 'Bot Abandonado'

]]


In [49]:
# --- ETAPA : Criar merge com base no Lojas ---

df_propostas_ajustado = pd.merge(df_propostas, df_lojas,
                                 on="Nome PDV", how="left", indicator=True)

# Substituir coluna 'Nome PDV' por 'Nome ajustado'
df_propostas_ajustado['Nome PDV'] = df_propostas_ajustado['Nome ajustado']

In [50]:
df_propostas_ajustado.columns

Index(['ID Lead', 'Data', 'Data ultima interação', 'Nome PDV', 'Nome Cliente',
       'Email Cliente', 'Telefone Cliente', 'CPF', 'Lead', 'Origem', 'Canal',
       'CallCenter status', 'Data Status Loja', 'Nome Vendedor',
       'ID do Vendedor', 'Callcenter detalhe', 'callcenter status detalhe',
       'assistente vendas', 'ID do Assistente de Vendas', 'tipo',
       'qualificado', 'documentos', 'tags', 'vetor_id',
       'data_ult_solicitacao_agendamento', 'data_ult_agendamento',
       'agendamento_visita', 'atraso_ult_agendamento', 'qtd_agendamentos_90d',
       'qtd_nao_foi_90d', 'regiao', 'campanha', 'Canal 2', 'Bot Abandonado',
       'CNPJ', 'Campanhas Facebook', 'Nome ajustado', 'Região', 'Varejo', 'UF',
       'Cidade', 'DDD', 'Geolocalização', 'Feirão 1 (Junho)',
       'Feirão 2 (Julho)', 'Feirão 3 (Agosto)', 'Feirão 4 (Fev 2025)',
       'Status', 'CEP', 'Telefone', 'Endereço ', 'Bairro', 'Mobi ID',
       'CNPJ DE', 'CNPJ PARA', 'Regional', '_merge'],
      dtype='object'

In [51]:
# Manter colunas uteis
df_propostas_ajustado = df_propostas_ajustado[['ID Lead', 'Data', 'Data ultima interação', 'Nome PDV', 'Nome Cliente',
       'Email Cliente', 'Telefone Cliente', 'CPF', 'Lead', 'Origem', 'Canal',
       'CallCenter status', 'Data Status Loja', 'Nome Vendedor',
       'ID do Vendedor', 'Callcenter detalhe', 'callcenter status detalhe',
       'assistente vendas', 'ID do Assistente de Vendas', 'tipo',
       'qualificado', 'documentos', 'tags', 'vetor_id',
       'data_ult_solicitacao_agendamento', 'data_ult_agendamento',
       'agendamento_visita', 'atraso_ult_agendamento', 'qtd_agendamentos_90d',
       'qtd_nao_foi_90d', 'regiao', 'campanha', 'Canal 2', 'Bot Abandonado',
       'CNPJ', 'Campanhas Facebook', 'Nome ajustado', 'Região', 'Varejo', 'UF',
       'Cidade', 'DDD', 'Regional', '_merge']]

In [52]:
# --- ETAPA : Criar arquivo csv e download ---

# Criar arquivo .csv
df_propostas_ajustado.to_csv("propostas_ajustado.csv",sep=";" ,index=False)

# Download do arquivo
from google.colab import files
files.download("propostas_ajustado.csv")

# Vizualização para conferência
df_propostas_ajustado.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,ID Lead,Data,Data ultima interação,Nome PDV,Nome Cliente,Email Cliente,Telefone Cliente,CPF,Lead,Origem,...,CNPJ,Campanhas Facebook,Nome ajustado,Região,Varejo,UF,Cidade,DDD,Regional,_merge
0,55410293,01/06/2025,01/06/2025,Seminovos Movida Indaiatuba,.,NaN,19971508681,NaN,NaN,MobiBot,...,21314559022640,lbtm_movida_seminovos_conversao_aon_facebook_a...,Seminovos Movida Indaiatuba,R04 - SPI2,Varejo 3,SP,Indaiatuba,19,Leonardo Alves Cardoso,both
1,55410294,01/06/2025,17/06/2025,Seminovos Movida Belo Horizonte Raja,Eliana,NaN,3599018963,NaN,NaN,MobiBot,...,21314559004900,NaN,Seminovos Movida Belo Horizonte Raja,R07 - MG,Varejo 1,MG,Belo Horizonte,31,Ari Gomes De Oliveira,both
2,55410295,01/06/2025,01/06/2025,Seminovos Movida Várzea Grande,NaN,NaN,6699407893,NaN,NaN,MobiBot,...,21314559018375,NaN,Seminovos Movida Várzea Grande,R08 - CO,Varejo 1,MT,Cuiabá,65,Kaique Silva Gorayeb,both
3,55410296,01/06/2025,01/06/2025,Seminovos Movida Recife Espinheiro,,NaN,8195383274,NaN,NaN,MobiBot,...,21314559011281,NaN,Seminovos Movida Recife Espinheiro,R09 - NORDESTE,Varejo 1,PE,Recife,81,Wellington Valfrido De Santana,both
4,55410297,01/06/2025,01/06/2025,Seminovos Movida João Pessoa Brisamar,Maicon,NaN,8387747181,NaN,NaN,MobiBot,...,21314559010986,NaN,Seminovos Movida João Pessoa Brisamar,R09 - NORDESTE,Varejo 1,PB,João Pessoa,83,Wellington Valfrido De Santana,both


In [35]:
print(df_propostas_ajustado.columns)

Index(['ID Lead', 'Data', 'Data ultima interação', 'Nome PDV', 'Nome Cliente',
       'Email Cliente', 'Telefone Cliente', 'CPF', 'Lead', 'Origem', 'Canal',
       'CallCenter status', 'Data Status Loja', 'Nome Vendedor',
       'ID do Vendedor', 'Callcenter detalhe', 'callcenter status detalhe',
       'assistente vendas', 'ID do Assistente de Vendas', 'tipo',
       'qualificado', 'documentos', 'tags', 'vetor_id',
       'data_ult_solicitacao_agendamento', 'data_ult_agendamento',
       'agendamento_visita', 'atraso_ult_agendamento', 'qtd_agendamentos_90d',
       'qtd_nao_foi_90d', 'regiao', 'campanha', 'Canal 2', 'Bot Abandonado',
       'CNPJ', 'Campanhas Facebook', 'Nome ajustado', 'Região', 'Varejo', 'UF',
       'Cidade', 'DDD', 'Regional', '_merge'],
      dtype='object')


In [36]:
df_propostas_ajustado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391194 entries, 0 to 391193
Data columns (total 44 columns):
 #   Column                            Non-Null Count   Dtype   
---  ------                            --------------   -----   
 0   ID Lead                           391194 non-null  int64   
 1   Data                              391194 non-null  object  
 2   Data ultima interação             391194 non-null  object  
 3   Nome PDV                          391194 non-null  object  
 4   Nome Cliente                      387770 non-null  object  
 5   Email Cliente                     243051 non-null  object  
 6   Telefone Cliente                  390938 non-null  object  
 7   CPF                               10852 non-null   float64 
 8   Lead                              51898 non-null   object  
 9   Origem                            391194 non-null  object  
 10  Canal                             376404 non-null  object  
 11  CallCenter status                 24146

In [ ]:
df_propostas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388306 entries, 0 to 388305
Data columns (total 37 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ID Lead                           388306 non-null  int64  
 1   Data                              388306 non-null  object 
 2   Data ultima interação             388306 non-null  object 
 3   Nome PDV                          388306 non-null  object 
 4   CNPJ PDV                          388306 non-null  int64  
 5   Nome Cliente                      384902 non-null  object 
 6   Email Cliente                     241314 non-null  object 
 7   Telefone Cliente                  388052 non-null  object 
 8   CPF                               10725 non-null   float64
 9   Lead                              51381 non-null   object 
 10  Origem                            388306 non-null  object 
 11  Canal                             373529 non-null  o

In [ ]:
df_propostas.columns

Index(['ID Lead', 'Data', 'Data ultima interação', 'Nome PDV', 'CNPJ PDV',
       'Nome Cliente', 'Email Cliente', 'Telefone Cliente', 'CPF', 'Lead',
       'Origem', 'Canal', 'Status', 'CallCenter status', 'Data Status Loja',
       'Nome Vendedor', 'ID do Vendedor', 'Callcenter detalhe',
       'callcenter status detalhe', 'assistente vendas',
       'ID do Assistente de Vendas', 'tipo', 'qualificado', 'documentos',
       'tags', 'vetor_id', 'data_ult_solicitacao_agendamento',
       'data_ult_agendamento', 'agendamento_visita', 'atraso_ult_agendamento',
       'qtd_agendamentos_90d', 'qtd_nao_foi_90d', 'regiao', 'Cidade', 'UF',
       'CEP', 'campanha', 'Bot Abandonado'],
      dtype='object')

In [ ]:
# --- ETAPA : Criar coluna de data ---

# Converter a coluna Data para datetime
df_propostas_ajustado["Data"] = pd.to_datetime(df_propostas_ajustado["Data"], format="%d/%m/%Y")

In [ ]:
df_propostas.head()

,ID Lead,Data,Data ultima interação,Nome PDV,CNPJ PDV,Nome Cliente,Email Cliente,Telefone Cliente,CPF,Lead,...,data_ult_agendamento,agendamento_visita,atraso_ult_agendamento,qtd_agendamentos_90d,qtd_nao_foi_90d,regiao,Cidade,UF,CEP,campanha
0,55410293,01/06/2025,01/06/2025,SEMINOVOS MOVIDA INDAIATUBA,21314559022640,.,NaN,19971508681,NaN,NaN,...,NaN,False,NaN,NaN,NaN,R03 – SPC2,NaN,NaN,NaN,NaN
1,55410294,01/06/2025,17/06/2025,SEMINOVOS MOVIDA RAJA,21314559004900,ELIANA,NaN,3599018963,NaN,NaN,...,NaN,False,NaN,NaN,NaN,R07 – MG,NaN,NaN,NaN,NaN
2,55410295,01/06/2025,01/06/2025,SEMINOVOS MOVIDA VÁRZEA GRANDE,21314559018375,NaN,NaN,6699407893,NaN,NaN,...,NaN,False,NaN,NaN,NaN,R08 – CENTRO OESTE,NaN,NaN,NaN,NaN
3,55410296,01/06/2025,01/06/2025,SEMINOVOS MOVIDA ESPINHEIRO,21314559011281,,NaN,8195383274,NaN,NaN,...,NaN,False,NaN,NaN,NaN,R09 – NORDESTE,NaN,NaN,NaN,NaN
4,55410297,01/06/2025,01/06/2025,SEMINOVOS MOVIDA JOÃO PESSOA BRISAMAR,21314559010986,MAICON,NaN,8387747181,NaN,NaN,...,NaN,False,NaN,NaN,NaN,R09 – NORDESTE,NaN,NaN,NaN,NaN
